In [1]:
import dask.dataframe as dd
import dask.bag as db
import json
import pandas as pd

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [2]:
!PowerShell.exe gc "./data/yt_metadata_en.jsonl" -head 2

{"categories": "Film & Animation", "channel_id": "UCzWrhkg9eK5I8Bm3HfV-unA", "crawl_date": "2019-10-31 20:19:26.270363", "description": "Lego City Police Lego Firetruck Cartoons about Lego City Movie for kids with a nice long video to keep the kids entertained while learning and having fun with the Lego City characters from Lego City undercover. Thanks for watching!", "dislike_count": 1.0, "display_id": "SBqSc91Hn9g", "duration": 1159, "like_count": 8.0, "tags": "lego city,lego police,lego city police,lego city episodes,videos de lego city,lego policia,lego bomberos,lego fire truck,lego firetruck,lego police chase,lego robbers,lego cartoons,lego movies,lego videos for kids", "title": "Lego City Police Lego Firetruck Cartoons about Lego City Movie for kids Episodes", "upload_date": "2016-09-28 00:00:00", "view_count": 1057.0}
{"categories": "Film & Animation", "channel_id": "UCzWrhkg9eK5I8Bm3HfV-unA", "crawl_date": "2019-10-31 20:19:26.914516", "description": "Lego Marvel SuperHeroes Le

In [4]:
FILE = "data/yt_metadata_en.jsonl"
OUTPUT_FILE = "data/yt_metadata_en.csv"


dtypes = {
    'categorie': 'str',                           
    'channel_id': 'str',                           
    'crawl_date': 'str',                           
    'dislike_count': 'uint32',                           
    'display_id': 'str',                           
    'duration': 'uint32',                           
    'like_count': 'uint32',                           
    'tags': 'str',                           
    'title': 'str',                           
    'upload_date': 'str',                           
    'view_count': 'uint64'                           
}

In [21]:
df = spark.read.json(FILE)

In [22]:
df.printSchema()

root
 |-- categories: string (nullable = true)
 |-- channel_id: string (nullable = true)
 |-- crawl_date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- dislike_count: double (nullable = true)
 |-- display_id: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- like_count: double (nullable = true)
 |-- tags: string (nullable = true)
 |-- title: string (nullable = true)
 |-- upload_date: string (nullable = true)
 |-- view_count: double (nullable = true)



In [29]:
df.show(2, vertical=True)

-RECORD 0-----------------------------
 categories    | Film & Animation     
 channel_id    | UCzWrhkg9eK5I8Bm3... 
 crawl_date    | 2019-10-31 20:19:... 
 description   | Lego City Police ... 
 dislike_count | 1.0                  
 display_id    | SBqSc91Hn9g          
 duration      | 1159                 
 like_count    | 8.0                  
 tags          | lego city,lego po... 
 title         | Lego City Police ... 
 upload_date   | 2016-09-28 00:00:00  
 view_count    | 1057.0               
-RECORD 1-----------------------------
 categories    | Film & Animation     
 channel_id    | UCzWrhkg9eK5I8Bm3... 
 crawl_date    | 2019-10-31 20:19:... 
 description   | Lego Marvel Super... 
 dislike_count | 1.0                  
 display_id    | UuugEl86ESY          
 duration      | 2681                 
 like_count    | 23.0                 
 tags          | Lego superheroes,... 
 title         | Lego Marvel Super... 
 upload_date   | 2016-09-28 00:00:00  
 view_count    | 12894.0 

In [31]:
df.select("dislike_count", "duration", "like_count", "view_count").summary().show()

+-------+------------------+------------------+------------------+------------------+
|summary|     dislike_count|          duration|        like_count|        view_count|
+-------+------------------+------------------+------------------+------------------+
|  count|          71293336|          72924794|          71293336|          72924140|
|   mean|  76.0915862037933| 757.6761375561788|1306.7580320831107|125237.60820707656|
| stddev|3064.7580067517442|1820.1105064271235|20135.880453098183|  3534488.21672248|
|    min|               0.0|                 1|               0.0|               0.0|
|    25%|               0.0|               134|               6.0|             532.0|
|    50%|               2.0|               284|              43.0|            2925.0|
|    75%|              12.0|               697|             265.0|           17423.0|
|    max|       1.6916346E7|            360157|       3.5287325E7|     6.509934494E9|
+-------+------------------+------------------+-------

In [20]:
df.count()

1

In [33]:
ddf = spark.read.option("multiline", "true").json(FILE)